### Imports

In [ ]:
import psycopg2
import pandas as pd
import os
import warnings

### Not doing anything complex to be bothered with SQLAlchemy

In [2]:
warnings.filterwarnings("ignore", message="pandas only supports SQLAlchemy")

### Connection parameters

In [3]:
DB_HOST = os.getenv('DB_HOST')
DB_PORT = os.getenv('DB_PORT')
DB_NAME = os.getenv('DB_NAME')
DB_USER = os.getenv('DB_USER')
DB_PASS = os.getenv('DB_PASS')

### Connect to the db

In [4]:
conn = psycopg2.connect(
    host=DB_HOST,
    port=DB_PORT,
    user=DB_USER,
    password=DB_PASS,
    dbname=DB_NAME
)

cur = conn.cursor()

### Create the requested tables

In [5]:
cur.execute(
    """
    CREATE TABLE departments (
        id SERIAL PRIMARY KEY,
        name VARCHAR(50) NOT NULL
    )
    """
)

cur.execute(
    """
    CREATE TABLE employees (
        id SERIAL PRIMARY KEY,
        name VARCHAR(50) NOT NULL,
        department_id INT REFERENCES departments(id)
    )
    """
)

conn.commit()

### Check if sample data is already there

In [6]:
pd.read_sql(
    """
    SELECT e.id, e.name, d.name as department
    FROM employees e
    JOIN departments d ON e.department_id = d.id
    """,
    conn
)

,id,name,department


### Insert sample data into the _departments_

In [7]:
cur.execute(
    "INSERT INTO departments (name) VALUES (%s), (%s), (%s) RETURNING id", 
    ("HR", "IT", "Finance")
)
department_ids = [row[0] for row in cur.fetchall()]

### Insert sample data into the _employees_

In [8]:
cur.execute(
    """
    INSERT INTO employees (name, department_id) VALUES
    (%s, %s), (%s, %s), (%s, %s), (%s, %s)
    """,
    (
        "Alice", department_ids[0],
        "Bob", department_ids[1],
        "Charlie", department_ids[1],
        "Diana", department_ids[2]
    )
)

conn.commit()

### Query the inserted sample data

In [9]:
pd.read_sql(
    """
    SELECT e.id, e.name, d.name as department
    FROM employees e
    JOIN departments d ON e.department_id = d.id
    """,
    conn
)

,id,name,department
0,1,Alice,HR
1,2,Bob,IT
2,3,Charlie,IT
3,4,Diana,Finance


### Cleanup

In [10]:
cur.close()
conn.close()